In [14]:
import numpy as np
import tensorflow as tf
import random as rn
import numpy as np

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
# Assume that you have 12GB of GPU memory and want to allocate ~4GB: 
# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
session_conf.gpu_options.allow_growth = True # 탄력적으로 memory 사용

from keras import backend as K

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras.layers import Input, Dense,LSTM,RepeatVector,GRU,Dropout,Reshape
from keras.layers import*
from keras.models import Model
from keras.models import Sequential
# from deap import base, creator, tools, algorithms
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import matplotlib.pyplot as plt
from math import sqrt
import random
import warnings
warnings.simplefilter("ignore", DeprecationWarning)


def parser(x):
	return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return Series(diff)

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[:,0][-interval]

# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = []
	for x in X:
   		 new_row = new_row+[i for i in x]
            
	new_row.append(value) 
	new_row_2 = np.array(new_row)
	new_row_2 = new_row_2.reshape((1,new_row_2.shape[0]))
	inverted = scaler.inverse_transform(new_row_2)
	return inverted[0, -1]


def create_dataset(dataset,features, look_back=1):
	dataset = np.insert(dataset,[0]*look_back,0)    
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back)]
		dataX.append(a)
		dataY.append(dataset[i + look_back])
	dataY= np.array(dataY)        
	dataY = np.reshape(dataY,(dataY.shape[0],features))
	dataset = np.concatenate((dataX,dataY),axis=1)  
	return dataset



# convert series to supervised learning
def series_to_supervised(data,features, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	print(f"n_vars = {n_vars}")
	x = np.zeros(features)
	print(f"x = {x}")
	print(f"data.shape is {data.shape}")
	for i in range(n_in):
		data = np.insert(data,x,0)
		print(f"{i}... insert x={x} to data at 0 index")
		print(f"\tdata.shape is {data.shape}")
		print(f"\tdata[{features}*{i}:{features}*{i}+{n_in}] = "
              f"{data[features*i:features*i+n_in]}")
	data = data.reshape(int(data.shape[0]/features),features) 
	print(f"data reshape to {data.shape}")
	df = DataFrame(data)
	cols, names = list(), list()
	print("Generaget cols and it's names...")
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg


# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(batch_size, X.shape[0], X.shape[1])
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]


def read_data(): 

	
	window_size = 0
	features = 8


	series = read_csv('pollution.csv', header=0, index_col=0)	
	raw_values = series.values

	# integer encode wind direction
	encoder = LabelEncoder()
	raw_values[:,4] = encoder.fit_transform(raw_values[:,4])

	# transform data to be stationary
	diff = difference(raw_values, 1)


	dataset = diff.values
	dataset = create_dataset(dataset,features,window_size)


	return dataset,raw_values

In [15]:
data,raw_values = read_data()


space = {'seq_len':hp.choice('seq_len',[5,10,15,20,25,30]),
         'epochs_pre':hp.choice('epochs_pre',[i for i in range(50,1000)]),
         'epochs_finetune':hp.choice('epochs_finetune',[i for i in range(50,500)]),
         'units':hp.choice('units',[i for i in range(1,50)]),
         'dropout':hp.choice('dropout',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         'batch_size':hp.choice('batch_size',[73, 146, 219])}

In [16]:
data

array([[19, 1, 0.0, ..., 0.8900000000000001, 0, 0],
       [11, 4, -1.0, ..., 0.8899999999999997, 0, 0],
       [22, 4, 0.0, ..., 1.7900000000000005, 1, 0],
       ...,
       [0, 0, 0.0, ..., 4.9199999999999875, 0, 0],
       [-2, 0, -1.0, ..., 4.02000000000001, 0, 0],
       [4, 1, 1.0, ..., 3.1299999999999955, 0, 0]], dtype=object)

In [17]:
data.shape

(43799, 8)

In [18]:
raw_values

array([[129, -16, -4.0, ..., 1.79, 0, 0],
       [148, -15, -4.0, ..., 2.68, 0, 0],
       [159, -11, -5.0, ..., 3.57, 0, 0],
       ...,
       [10, -22, -3.0, ..., 242.7, 0, 0],
       [8, -22, -4.0, ..., 246.72, 0, 0],
       [12, -21, -3.0, ..., 249.85, 0, 0]], dtype=object)

In [19]:
raw_values.shape

(43800, 8)

In [20]:
space = {'seq_len':hp.choice('seq_len',[5,10,15,20,25,30]),
         'epochs_pre':hp.choice('epochs_pre',[i for i in range(50,1000)]),
         'epochs_finetune':hp.choice('epochs_finetune',[i for i in range(50,500)]),
         'units':hp.choice('units',[i for i in range(1,50)]),
         'dropout':hp.choice('dropout',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]),
         'batch_size':hp.choice('batch_size',[73, 146, 219])}

In [21]:
params = {'seq_len': 10,
          'epochs_pre': 100,
          'epochs_finetune': 50,
          'units': 20,
          'dropout': 0.2,
          'batch_size': 73}

In [22]:
features = 8

print ('Params testing: ', params)

# frame as supervised learning
reframed = series_to_supervised(data,features, params['seq_len'], 1)
drop = [i for  i in  range(params['seq_len']*features+1,((params['seq_len']+1)*features))]
reframed.drop(reframed.columns[drop], axis=1, inplace=True)
reframed = reframed.values

Params testing:  {'seq_len': 10, 'epochs_pre': 100, 'epochs_finetune': 50, 'units': 20, 'dropout': 0.2, 'batch_size': 73}
n_vars = 8
x = [0. 0. 0. 0. 0. 0. 0. 0.]
data.shape is (43799, 8)
0... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350400,)
	data[8*0:8*0+10] = [0 0 0 0 0 0 0 0 19 1]
1... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350408,)
	data[8*1:8*1+10] = [0 0 0 0 0 0 0 0 19 1]
2... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350416,)
	data[8*2:8*2+10] = [0 0 0 0 0 0 0 0 19 1]
3... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350424,)
	data[8*3:8*3+10] = [0 0 0 0 0 0 0 0 19 1]
4... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350432,)
	data[8*4:8*4+10] = [0 0 0 0 0 0 0 0 19 1]
5... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at 0 index
	data.shape is (350440,)
	data[8*5:8*5+10] = [0 0 0 0 0 0 0 0 19 1]
6... insert x=[0. 0. 0. 0. 0. 0. 0. 0.] to data at

In [23]:
# split into train and test sets
train_size = 365*24*4
train, test = reframed[0:train_size], reframed[train_size:]


# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)

# divide train into train and valid
train2_size = int(365*24*3.5)
train_scaled2, valid = train_scaled[0:train2_size], train_scaled[train2_size:]

# split into input and outputs
x_train,y_train = train_scaled2[:,0:-1],train_scaled2[:,-1]
x_valid,y_valid = valid[:,0:-1],valid[:,-1]
x_test,y_test = test_scaled[:,0:-1],test_scaled[:,-1]

# reshape input to be 3D [samples, timesteps, features]
x_train = x_train.reshape(x_train.shape[0],params['seq_len'],features)
x_valid = x_valid.reshape(x_valid.shape[0],params['seq_len'],features)
x_test = x_test.reshape(x_test.shape[0],params['seq_len'],features)
print(x_train.shape, y_train.shape,x_valid.shape,y_valid.shape, x_test.shape, y_test.shape)

(30660, 10, 8) (30660,) (4380, 10, 8) (4380,) (8759, 10, 8) (8759,)


- First Block

In [24]:
print('start pretraining')
print('===============')

# train AE
timesteps = x_train.shape[1]
input_dim = x_train.shape[2]
AE1 = Sequential()
AE1.add(CuDNNLSTM(params['units'], batch_input_shape=(params['batch_size'], timesteps, input_dim), stateful=False))
AE1.add(RepeatVector(timesteps))
AE1.add(CuDNNLSTM(input_dim,stateful = False, return_sequences=True))

AE1.compile(loss='mean_squared_error', optimizer='Adam')

AE1.fit(x_train, x_train,
        epochs=params['epochs_pre'],
        batch_size= params['batch_size'],
        shuffle=True,
        verbose=1)

trained_encoder = AE1.layers[0]
weights = AE1.layers[0].get_weights()

start pretraining
Epoch 1/100
30660/30660 [==============================] - 2s 73us/step - loss: 0.0435
Epoch 2/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0208
Epoch 3/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0199
Epoch 4/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0194
Epoch 5/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0191
Epoch 6/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0190
Epoch 7/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0188
Epoch 8/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0188
Epoch 9/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0187
Epoch 10/100
30660/30660 [==============================] - 2s 64us/step - loss: 0.0186
Epoch 11/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0186
Epoch 12/100
30660/3066

30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 94/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 95/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 96/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 97/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 98/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 99/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178
Epoch 100/100
30660/30660 [==============================] - 2s 61us/step - loss: 0.0178


In [25]:
# Fine-turning
print('Fine-turning')
print('============')

#build finetuning model
model1 = Sequential()
model1.add(trained_encoder)
model1.layers[-1].set_weights(weights)
model1.add(Dropout(params['dropout']))
model1.add(Dense(1))

model1.compile(loss='mean_squared_error', optimizer='Adam')

model1.fit(x_train, y_train, epochs=params['epochs_finetune'], batch_size = params['batch_size'], verbose = 1,shuffle=True)

model1.save('layer1.h5')

Fine-turning
Epoch 1/50
30660/30660 [==============================] - 2s 52us/step - loss: 0.0442
Epoch 2/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0036
Epoch 3/50
30660/30660 [==============================] - 1s 46us/step - loss: 0.0017
Epoch 4/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0014
Epoch 5/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0012
Epoch 6/50
30660/30660 [==============================] - 1s 46us/step - loss: 0.0012
Epoch 7/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0012
Epoch 8/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0011
Epoch 9/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0011
Epoch 10/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0011
Epoch 11/50
30660/30660 [==============================] - 1s 45us/step - loss: 0.0011
Epoch 12/50
30660/30660 [==============

- Second Block

In [27]:
print('start pretraining')
print('===============')

# train AE
timesteps = x_train.shape[1]
input_dim = x_train.shape[2]
hidden_layers = [25, 26]
epochs_pre = [100, 100]
trained_encoder = []
x_train_temp = x_train
for i, (hidden, epochs) in enumerate(zip(hidden_layers, epochs_pre), start=1):
    print(f"pretrain Autoencoder: {i} ----> Encoder: {hidden} ----> Epochs: {epochs}")
    print(x_train_temp.shape)
    print('=============================================================')
    
    inputs  = Input(batch_shape=(params['batch_size'], timesteps, x_train_temp.shape[2]))
    encoded = CuDNNLSTM(hidden,
                        batch_input_shape=(params['batch_size'], timesteps, x_train_temp.shape[2]),
                        stateful = False)(inputs)
    decoded = RepeatVector(timesteps)(encoded)
    decoded = CuDNNLSTM(input_dim,
                        stateful=False,
                        return_sequences=True)(decoded)
    
    AE2 = Model(inputs, decoded)
    AE2.compile(loss='mean_squared_error', optimizer='Adam')
    
    AE2.fit(x_train_temp, x_train,
            epochs=epochs,
            batch_size= params['batch_size'],
            shuffle=True,
            verbose=1)
    
    # store trained encoder and its weights
    trained_encoder.append((AE2.layers[1], AE2.layers[1].get_weights()))
    
    # update training data
    x_train_temp = encoder.predict(x_train_temp, batch_size=params['batch_size'])
    
    # reshape encoded input ot 3D
    inputs = Input(shape=(x_train_temp.shape[1],))
    reshape = RepeatVector(timesteps)(inputs)
    Repeat = Model(inputs, reshape)
    
    x_train_temp = Repeat.predict(x_train_temp, batch_size=params['batch_size'])

start pretraining
pretrain Autoencoder: 1 ----> Encoder: 25 ----> Epochs: 100
(30660, 10, 8)
Epoch 1/100
30660/30660 [==============================] - 2s 78us/step - loss: 0.0351
Epoch 2/100
30660/30660 [==============================] - 2s 68us/step - loss: 0.0211
Epoch 3/100
30660/30660 [==============================] - 2s 67us/step - loss: 0.0204
Epoch 4/100
30660/30660 [==============================] - 2s 67us/step - loss: 0.0196
Epoch 5/100
30660/30660 [==============================] - 2s 67us/step - loss: 0.0194
Epoch 6/100
30660/30660 [==============================] - 2s 67us/step - loss: 0.0192
Epoch 7/100
30660/30660 [==============================] - 2s 67us/step - loss: 0.0191
Epoch 8/100
30660/30660 [==============================] - 2s 68us/step - loss: 0.0190
Epoch 9/100
30660/30660 [==============================] - 2s 63us/step - loss: 0.0188
Epoch 10/100
30660/30660 [==============================] - 2s 63us/step - loss: 0.0187
Epoch 11/100
30660/30660 [==========

30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 93/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 94/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 95/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 96/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 97/100
30660/30660 [==============================] - 2s 66us/step - loss: 0.0178
Epoch 98/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0178
Epoch 99/100
30660/30660 [==============================] - 2s 66us/step - loss: 0.0178
Epoch 100/100
30660/30660 [==============================] - 2s 65us/step - loss: 0.0177


NameError: name 'encoder' is not defined

In [ ]:
# Fine-tuning
epochs_finetune = 463
print('\nFine-tuning')
print('============')

l = len(trained_encoder)
# build finetuning model
model = Sequential()
for i, encod in enumerate(trained_encoder):
    model.add(encod[0])
    model.layers[-1].set_weights(encod[1])
    model.add(Dropout(dropout))
    if (i+1 != 1):
        model.add(RepeatVector(timesteps))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='Adam')
model.fit(x_train, y_train, epochs=epochs_finetune, )

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(np.abs(F - A) / (np.abs(A) + np.abs(F))